# 부산 지하철 승하차 승객수 분석

- 부산 지하철 주소 데이터 
- 승하차 승객수 

In [6]:
import pandas as pd

address = pd.read_csv('국가철도공단_부산_지하철_주소데이터_20221122.csv', encoding = 'cp949')
address

,철도운영기관명,선명,역명,지번주소,도로명주소
0,부산교통공사,1호선,괴정,부산광역시 사하구 괴정동 934-3,부산광역시 사하구 낙동대로 지하 221(괴정동)
1,부산교통공사,1호선,교대,부산광역시 연제구 거제동 72-25,부산광역시 연제구 중앙대로 지하 1217
2,부산교통공사,1호선,구서,부산광역시 금정구 구서동 475,부산광역시 금정구 금정로 233번길 46(구서동)
3,부산교통공사,1호선,남산,부산광역시 금정구 남산동 136-1,부산광역시 금정구 중앙대로 2019-1
4,부산교통공사,1호선,남포,부산광역시 중구 남포동1가 11,부산광역시 중구 구덕로 지하 12
...,...,...,...,...,...
109,부산교통공사,4호선,수안,부산광역시 동래구 수안동 204-2,부산광역시 동래구 충렬대로 지하 223
110,부산교통공사,4호선,안평,부산광역시 기장군 철마면 안평리 543-23,부산광역시 기장군 철마면 반송로 1001
111,부산교통공사,4호선,영산대,부산광역시 해운대구 반송동 698-12,부산광역시 해운대구 반송로 803
112,부산교통공사,4호선,윗반송,부산광역시 해운대구 반송동 385-56,부산광역시 해운대구 반송로 917


## 위도 경도 가져오기

In [7]:
import requests

api_key = ''
header = {'Authorization': 'KakaoAK ' + api_key}

url_header = 'https://dapi.kakao.com/v2/local/search/address.json?query='

def getGeoCoder(address):
    result = ''    
    
    url = url_header + address    
    r = requests.get(url, headers= header)
    
    if r.status_code == 200:
        try:
            result_address = r.json()['documents'][0]['address']
            result = result_address['y'], result_address['x']
        except Exception as err:
            return None
    else:
        result = 'ERROR[' + str(r.status_code) + ']'
    
    return result

In [11]:
df = pd.read_csv('일별 역별 시간대별 승하차(2023년 4월).csv', encoding = 'cp949', usecols = ['역번호', '역명','합계'])

,역번호,역명,합계
0,95,다대포해수욕장,6311
1,95,다대포해수욕장,5839
2,95,다대포해수욕장,4264
3,95,다대포해수욕장,3300
4,95,다대포해수욕장,4431


In [12]:
df['합계'] = df['합계'].str.replace(',','').astype(int)
df['역명'] = df['역명'].str.replace(' ','')

In [16]:
df['역명'] = df['역명'].str.replace('\d+', '', regex=True)

In [17]:
df.info()
df['역명'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26880 entries, 0 to 26879
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   역번호     26880 non-null  int64 
 1   역명      26880 non-null  object
 2   합계      26880 non-null  int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 525.1+ KB


array(['다대포해수욕장', '다대포항', '낫개', '신장림', '장림', '동매', '신평', '하단', '당리', '사하',
       '괴정', '대티', '서대신', '동대신', '토성', '자갈치', '남포', '중앙', '부산역', '초량',
       '부산진', '좌천', '범일', '범내골', '서면', '부전', '양정', '시청', '연산', '교대', '동래',
       '명륜', '온천장', '부산대', '장전', '구서', '두실', '남산', '범어사', '노포', '장산',
       '중동', '해운대', '동백', '벡스코', '센텀시티', '민락', '수영', '광안', '금련산', '남천',
       '경성대부경대', '대연', '못골', '지게골', '문현', '국제금융센터', '전포', '부암', '가야',
       '동의대', '개금', '냉정', '주례', '감전', '사상', '덕포', '모덕', '모라', '구남', '구명',
       '덕천', '수정', '화명', '율리', '동원', '금곡', '호포', '증산', '부산대양산', '남양산',
       '양산', '망미', '배산', '물만골', '거제', '종합운동장', '사직', '미남', '만덕', '남산정',
       '숙등', '구포', '강서구청', '체육공원', '대저', '수안', '낙민', '충렬사', '명장', '서동',
       '금사', '반여농산물', '석대', '영산대', '윗반송', '고촌', '안평'], dtype=object)

In [18]:
def get_line(station_no):
    if station_no < 200:
        return '1호선'
    elif station_no < 300:
        return '2호선'
    elif station_no < 400:
        return '3호선'
    else:
        return '4호선'

In [19]:
df['호선'] = df['역번호'].apply(get_line)

## 승객수와 주소 테이블을 병합

In [31]:
passenger = pd.DataFrame(df.groupby(['호선','역명'])['합계'].mean().astype(int))
address = address.rename(columns={'선명':'호선'})

In [33]:
merge_df = pd.merge(passenger, address, on =['호선','역명'], how='left')

In [34]:
merge_df

,호선,역명,합계,철도운영기관명,지번주소,도로명주소
0,1호선,괴정,8539,부산교통공사,부산광역시 사하구 괴정동 934-3,부산광역시 사하구 낙동대로 지하 221(괴정동)
1,1호선,교대,13444,부산교통공사,부산광역시 연제구 거제동 72-25,부산광역시 연제구 중앙대로 지하 1217
2,1호선,구서,9041,부산교통공사,부산광역시 금정구 구서동 475,부산광역시 금정구 금정로 233번길 46(구서동)
3,1호선,남산,5247,부산교통공사,부산광역시 금정구 남산동 136-1,부산광역시 금정구 중앙대로 2019-1
4,1호선,남포,17327,부산교통공사,부산광역시 중구 남포동1가 11,부산광역시 중구 구덕로 지하 12
...,...,...,...,...,...,...
107,4호선,수안,3326,부산교통공사,부산광역시 동래구 수안동 204-2,부산광역시 동래구 충렬대로 지하 223
108,4호선,안평,1940,부산교통공사,부산광역시 기장군 철마면 안평리 543-23,부산광역시 기장군 철마면 반송로 1001
109,4호선,영산대,2602,부산교통공사,부산광역시 해운대구 반송동 698-12,부산광역시 해운대구 반송로 803
110,4호선,윗반송,2354,부산교통공사,부산광역시 해운대구 반송동 385-56,부산광역시 해운대구 반송로 917


In [41]:
m = folium.Map(location = [35.179774, 129.075004], zoom_start = 12)

geotable = []
for index, row in merge_df.iterrows():
    #print(row['호선'], row['역명'], row['지번주소'], row['도로명주소'], row['합계'])
    geoinfo = getGeoCoder(row['지번주소'])
    
    if geoinfo is None:
            geoinfo = getGeoCoder(row['도로명주소'])
            
    geotable.append([geoinfo[0], geoinfo[1], row['합계']])

HeatMap(geotable).add_to(m)

m

# HeatMap sample

In [9]:
from folium.plugins import HeatMap
import folium

m = folium.Map(location=[35.19600417088537, 129.0815799388583],zoom_start=12)

heat_data = [
    [35.179774, 129.075004, 100],  # Example latitude and longitude values
    [35.180000, 129.075100, 10],
    [35.180200, 129.075300, 1]]

HeatMap(heat_data).add_to(m)

m
